In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

import matplotlib.pyplot as plt

In [ ]:
# device config

device = torch.device('cuda')

In [ ]:
# hyper parameters

input_size = 784
hidden_size = 100
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001

In [ ]:
train_data = torchvision.datasets.MNIST(root='./data', train=True, transform=transforms.ToTensor(), download=True)

In [ ]:
test_data = torchvision.datasets.MNIST(root='./data', train=False, transform=transforms.ToTensor(), download=True)

In [ ]:
train_loader = torch.utils.data.DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_data, batch_size=batch_size, shuffle=False)

In [ ]:
examples = iter(train_loader)
sample, label = examples.next()

print(sample.shape, label.shape)

torch.Size([100, 1, 28, 28]) torch.Size([100])


In [ ]:
class Neuralnet(nn.Module):
  def __init__(self, input_size, hidden_size, num_class):
    super(Neuralnet, self).__init__()
    self.l1 = nn.Linear(input_size, hidden_size)
    self.relu = nn.ReLU()
    self.l2 = nn.Linear(hidden_size, num_class)

  def forward(self, samples):
    out = self.l1(samples)
    out = self.relu(out)
    out = self.l2(out)
    return out


model = Neuralnet(input_size, hidden_size, num_classes)
model.to(device)

# loss and optimizer
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


#training loop

n_total_steps = len(train_loader)

for epoch in range(num_epochs):
  for i, (images, labels) in enumerate(train_loader):
    # 100, 1, 28, 28
    # 100, 784
    images = images.reshape(-1, 28*28).to(device)
    labels = labels.to(device)

    #forward
    outputs = model(images)
    loss = loss_func(outputs, labels)

    # backwards
    optimizer.zero_grad() # empty the graidents
    loss.backward() # process the backpropagation
    optimizer.step() # update the weights

    if (i + 1) % 100 == 0:
      print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')

with torch.no_grad():
  n_corr = 0
  n_samp = 0
  for images, labels in test_loader:
    images = images.reshape(-1, 28*28).to(device)
    labels = labels.to(device)
    outputs = model(images)

    #value, index
    _, predictions = torch.max(outputs, 1)
    n_samp += labels.shape[0]
    n_corr = (predictions == labels).sum().item()

  acc = 100.0 * n_corr / n_samp 
  print("accuracy:", acc)

Epoch [1/3], Step [100/600], Loss: 0.4438
Epoch [1/3], Step [200/600], Loss: 0.2700
Epoch [1/3], Step [300/600], Loss: 0.2818
Epoch [1/3], Step [400/600], Loss: 0.3984
Epoch [1/3], Step [500/600], Loss: 0.1943
Epoch [1/3], Step [600/600], Loss: 0.1980
Epoch [2/3], Step [100/600], Loss: 0.2226
Epoch [2/3], Step [200/600], Loss: 0.1768
Epoch [2/3], Step [300/600], Loss: 0.1205
Epoch [2/3], Step [400/600], Loss: 0.1860
Epoch [2/3], Step [500/600], Loss: 0.1354
Epoch [2/3], Step [600/600], Loss: 0.2614
Epoch [3/3], Step [100/600], Loss: 0.1446
Epoch [3/3], Step [200/600], Loss: 0.1819
Epoch [3/3], Step [300/600], Loss: 0.1918
Epoch [3/3], Step [400/600], Loss: 0.1411
Epoch [3/3], Step [500/600], Loss: 0.1738
Epoch [3/3], Step [600/600], Loss: 0.0765
accuracy: 0.95
